# ARIMA와 SARIMA 모델을 이용한 시계열 예측

이 노트북은 ARIMA(AutoRegressive Integrated Moving Average)와 SARIMA(Seasonal ARIMA) 모델을 이용하여 시계열 데이터를 분석하고 예측하는 과정을 보여줍니다.
ARIMA 모델은 시계열 데이터의 자기상관성(Autocorrelation)과 이동평균(Moving Average) 특성을 활용하며, 비정상성(Non-stationary) 데이터를 차분(Differencing)을 통해 정상성으로 변환하여 모델링합니다.
SARIMA 모델은 ARIMA 모델에 계절성(Seasonality) 요인을 추가하여 주기적인 패턴을 가진 시계열 데이터 분석에 더 적합합니다.

## 사용 가이드
1. **데이터 설정**: `DATA_PATH`, `DATE_COL`, `TARGET_COL` 변수를 실제 데이터에 맞게 수정합니다.
2. **모델 파라미터**: `P`, `D`, `Q`, `SEASONAL`, `SEASONAL_P`, `SEASONAL_D`, `SEASONAL_Q`, `SEASONAL_PERIOD` 등의 파라미터를 데이터 특성에 맞게 조정합니다.
3. **실행**: 노트북 셀을 순차적으로 실행하며 각 단계의 결과와 설명을 확인합니다.

## 분석 흐름
1. **라이브러리 임포트 및 설정**: 분석에 필요한 라이브러리를 불러오고 기본 설정을 수행합니다.
2. **데이터 로드 및 탐색**: 지정된 경로에서 데이터를 로드하고, 데이터의 기본적인 구조와 통계 정보를 확인하여 데이터에 대한 초기 이해를 돕습니다.
3. **시계열 데이터 전처리**: 날짜 컬럼을 인덱스로 설정하고, 타겟 변수를 선택합니다. 결측치를 처리하고 필요시 리샘플링을 수행합니다.
4. **시계열 데이터 시각화**: 원본 데이터와 분포를 시각화하여 데이터의 패턴과 특성을 직관적으로 파악합니다.
5. **정상성 확인 및 변환**: ACF/PACF 분석과 통계적 검정(ADF)을 통해 정상성을 확인하고, 필요시 차분을 통해 정상성을 확보합니다.
6. **모델 파라미터 선택**: ACF/PACF 분석 결과나 Auto ARIMA를 통해 최적의 모델 파라미터를 결정합니다.
7. **모델 구축 및 학습**: 선택된 파라미터로 ARIMA 또는 SARIMA 모델을 구축하고 학습합니다.
8. **모델 예측 및 평가**: 학습된 모델로 훈련/테스트 데이터 및 미래 기간에 대한 예측을 수행하고 결과를 평가합니다.
9. **결론 및 인사이트**: 분석 결과와 모델 성능을 요약하고, 데이터에 대한 인사이트를 도출합니다.

## 1. 사용자 입력 파라미터 설정
분석 대상 데이터와 모델 파라미터를 사용자의 환경과 목적에 맞게 설정합니다.

In [ ]:
# ===== 데이터 파라미터 =====
# 데이터 파일 경로 (CSV 형식 권장)
DATA_PATH = '../data/raw/your_data.csv'  # 실제 데이터 파일 경로를 지정하세요

# 시간 열 이름 (데이터프레임 내 날짜/시간 정보가 있는 열)
DATE_COL = 'date'  # 실제 데이터의 날짜 열 이름을 지정하세요

# 타겟 열 이름 (예측하려는 변수가 있는 열)
TARGET_COL = 'value'  # 실제 데이터의 타겟 열 이름을 지정하세요

# 데이터 분할 비율
TRAIN_SIZE = 0.8                         # 훈련 데이터 비율 (8:2 분할)

# 시계열 데이터 주기 설정
FREQUENCY = 'D'  # D: 일별, M: 월별, Y: 연별, H: 시간별 등 (pandas frequency 문자열)

# ===== ARIMA 모델 파라미터 =====
# Auto ARIMA 사용 여부
USE_AUTO_ARIMA = False # True로 설정 시 아래 p, d, q, P, D, Q 값 대신 auto_arima가 찾은 최적 파라미터 사용

# ARIMA 파라미터 (비계절성)
P = 1  # AR 차수
D = 1  # 차분 차수
Q = 1  # MA 차수

# ===== SARIMA 모델 파라미터 =====
# 계절성 사용 여부
SEASONAL = True

# 계절성 ARIMA 파라미터
SEASONAL_P = 1  # 계절성 AR 차수
SEASONAL_D = 1  # 계절성 차분 차수
SEASONAL_Q = 1  # 계절성 MA 차수
SEASONAL_PERIOD = 12 # 계절성 주기

# ===== 예측 파라미터 =====
# 미래 예측 기간
FORECAST_HORIZON = 12  # 미래 예측 스텝 수

# ===== 출력 경로 설정 =====
# 결과 저장 경로
OUTPUT_DIR = '../plots'  # 시각화, 모델 결과를 저장할 경로
DATA_OUTPUT_DIR = '../data/processed'  # 데이터 파일을 저장할 경로

## 2. 필요 라이브러리 임포트
시계열 분석, 모델링, 시각화 등에 필요한 라이브러리를 불러오고 기본 설정을 수행합니다.

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats

# 시계열 분석 라이브러리
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pmdarima as pm  # auto_arima 사용을 위해 추가

# 경고 무시
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

# 한글 폰트 설정 (필요한 경우)
try:
    plt.rc('font', family='Malgun Gothic')
    plt.rc('axes', unicode_minus=False)
except:
    pass

## 3. 데이터 로드 및 탐색
지정된 경로에서 데이터를 로드하고, 데이터의 기본적인 구조와 통계 정보를 확인하여 데이터에 대한 초기 이해를 돕습니다.

In [ ]:
# 데이터 로드
print(f"데이터 파일 '{DATA_PATH}'을 로드합니다...")
df = pd.read_csv(DATA_PATH)
print(f"데이터 로드 완료: {df.shape[0]}행 {df.shape[1]}열")

# 데이터 정보 확인
print("\n데이터 정보:")
print(df.info())

print("\n데이터 통계:")
print(df.describe())

print("\n데이터 샘플:")
print(df.head())

## 4. 시계열 데이터 전처리
시계열 분석에 적합하도록 데이터를 가공합니다.
- **날짜 인덱스 설정**: 시간 순서를 기준으로 데이터를 분석하기 위해 날짜/시간 컬럼을 인덱스로 설정합니다.
- **정렬**: 시간 순서가 올바른지 확인하고 정렬합니다.
- **타겟 변수 추출**: 분석하고 예측하려는 주요 변수(컬럼)를 선택합니다.
- **결측치 처리**: 필요시 보간하여 모델링에 영향을 미치는 결측값을 처리합니다.

In [ ]:
# 날짜 열을 인덱스로 변환
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df.set_index(DATE_COL, inplace=True)
print(f"날짜 인덱스 변환 완료: {df.index.min()} ~ {df.index.max()}")

# 날짜순으로 정렬
df.sort_index(inplace=True)

# 선택한 타겟 변수 추출
ts_data = df[TARGET_COL].copy()
print(f"타겟 변수 '{TARGET_COL}' 추출 완료, 데이터 포인트: {len(ts_data)}")

### 4.1 결측치 처리
시계열 데이터에서 결측치는 분석 결과에 큰 영향을 미칠 수 있습니다. 결측치 처리는 데이터의 연속성을 유지하는 것이 중요합니다.

**시나리오 A**: 선형 보간법으로 결측치 처리
- 시계열의 연속성을 최대한 유지하면서 결측치를 채우는 방법입니다.
- 대부분의 시계열 분석에서 권장되는 기본 접근법입니다.

In [ ]:
# 결측치 확인 및 처리
missing_values = ts_data.isnull().sum()
print(f"결측치 개수: {missing_values}")

# 시나리오 A: 결측치가 있는 경우 - 선형 보간법으로 처리
# 시계열의 연속성을 최대한 유지하면서 결측치를 채우는 방법
# 대부분의 시계열 분석에서 권장되는 기본 접근법
ts_data = ts_data.interpolate(method='linear')
print("결측치 선형 보간 완료")
print(f"처리 후 결측치 개수: {ts_data.isnull().sum()}")

## 5. 시계열 데이터 시각화
변환된 시계열 데이터를 시각화하여 전반적인 패턴(추세, 계절성 등)과 분포를 확인합니다.
- **선 그래프**: 시간에 따른 타겟 변수의 변화 추이를 보여줍니다.
- **분포 플롯 (히스토그램, Q-Q 플롯)**: 타겟 변수 값의 분포 형태와 정규성 가정을 시각적으로 검토합니다.

In [ ]:
# 시계열 데이터 시각화
plt.figure(figsize=(14, 7))
plt.plot(ts_data, label=TARGET_COL)
plt.title(f'{TARGET_COL} 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.grid(True, alpha=0.3)
plt.legend()
plt.savefig(f'{OUTPUT_DIR}/arima_시계열데이터.png')
plt.show()

# 분포 확인
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
sns.histplot(ts_data, kde=True)
plt.title(f'{TARGET_COL} 분포')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
stats.probplot(ts_data, plot=plt)
plt.title('Q-Q 플롯')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_시계열_분포.png')
plt.show()

## 6. 정상성 확인 및 변환
ARIMA/SARIMA 모델은 입력 데이터가 정상성(Stationarity)을 만족할 때 더 안정적인 성능을 보입니다. 정상성이란 시계열의 통계적 특성(평균, 분산, 자기공분산)이 시간에 따라 변하지 않는 것을 의미합니다.
- **정상성 확인**: 
  - **ACF/PACF 플롯**: 자기상관함수(ACF)와 부분자기상관함수(PACF) 플롯을 통해 시계열의 자기상관 구조를 시각적으로 확인합니다.
  - **ADF 검정**: 통계적 검정을 통해 정상성 여부를 판단합니다. 귀무가설은 "단위근이 존재한다(=시계열이 비정상적이다)"이며, p-value가 0.05보다 작으면 귀무가설을 기각하여 정상성을 만족한다고 볼 수 있습니다.
- **정상성 변환**: 
  - **차분(Differencing)**: 비정상성 시계열을 정상성으로 변환하는 가장 일반적인 방법입니다. 현재 시점의 데이터에서 이전 시점의 데이터를 빼는 방식으로, 추세를 제거하는 효과가 있습니다.

In [ ]:
# ACF 및 PACF 플롯
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plot_acf(ts_data, lags=20, alpha=0.05, ax=plt.gca())
plt.title('자기상관함수 (ACF)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plot_pacf(ts_data, lags=20, alpha=0.05, method='ywm', ax=plt.gca())
plt.title('부분 자기상관함수 (PACF)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_ACF_PACF.png')
plt.show()

### 6.1 정상성 검정
시계열 데이터가 정상성을 만족하는지 통계적으로 검정합니다. ADF 검정에서 귀무가설은 "시계열이 비정상적이다"이며, 
p-value가 0.05보다 작으면 귀무가설을 기각하고 시계열이 정상적이라고 판단합니다.

In [ ]:
# ADF 검정 실행
adf_result = adfuller(ts_data.dropna())
print('ADF 검정 결과:')
print(f'ADF 통계량: {adf_result[0]:.4f}')
print(f'p-value: {adf_result[1]:.4f}')
print('임계값:')
for key, value in adf_result[4].items():
    print(f'\t{key}: {value:.4f}')

# 결과 해석 (p-value 기준)
print('\nADF 검정 결과 해석:')
if adf_result[1] < 0.05:
    print('귀무가설 기각 (p < 0.05): 시계열이 정상적일 가능성이 높습니다.')
    is_stationary = True
else:
    print('귀무가설 채택 (p >= 0.05): 시계열이 비정상적일 가능성이 높습니다.')
    is_stationary = False

### 6.2 일반 차분을 통한 정상성 확보
데이터가 비정상적일 경우, 차분을 적용하여 정상성을 확보할 수 있습니다. ARIMA/SARIMA 모델에서 d 파라미터는
이 차분 차수를 의미합니다. 현재 `D = 1`로 설정되어 있으며, 필요시 조정 가능합니다.

In [ ]:
# 일반 차분 적용
diff_data = ts_data.diff(D).dropna()
print(f"{D}차 차분 데이터 생성 완료")

# 차분 데이터 시각화
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(ts_data, label='원본 데이터')
plt.title('원본 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(diff_data, label=f'{D}차 차분')
plt.title(f'{D}차 차분 데이터')
plt.xlabel('날짜')
plt.ylabel(f'차분({TARGET_COL})')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_시계열_차분.png')
plt.show()

# 차분 데이터의 ACF 및 PACF 플롯
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plot_acf(diff_data, lags=20, alpha=0.05, ax=plt.gca())
plt.title(f'{D}차 차분 데이터의 ACF')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plot_pacf(diff_data, lags=20, alpha=0.05, method='ywm', ax=plt.gca())
plt.title(f'{D}차 차분 데이터의 PACF')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_차분_ACF_PACF.png')
plt.show()

# 차분 후 정상성 검정
adf_diff_result = adfuller(diff_data.dropna())
print(f'\n{D}차 차분 후 ADF 검정 결과:')
print(f'ADF 통계량: {adf_diff_result[0]:.4f}')
print(f'p-value: {adf_diff_result[1]:.4f}')

# 차분 후 결과 해석
if adf_diff_result[1] < 0.05:
    print('차분 후 시계열이 정상적일 가능성이 높습니다 (p < 0.05).')
    diff_is_stationary = True
else:
    print('차분 후에도 시계열이 비정상적일 가능성이 높습니다 (p >= 0.05). 추가 차분이 필요할 수 있습니다.')
    diff_is_stationary = False

### 6.3 계절성 차분 (SARIMA 모델용)
계절성이 있는 시계열 데이터의 경우, 계절성 차분을 통해 계절 패턴을 제거할 수 있습니다.
계절성 차분은 현재 값에서 한 계절(예: 12개월, 7일) 이전의 값을 뺍니다.

**시나리오 A**: 계절성 차분 적용
- 계절성이 강한 데이터에 적합하며, SARIMA 모델의 D 파라미터에 해당합니다.
- 현재 `SEASONAL=True`이고 `SEASONAL_D=1`로 설정되어 있으므로 계절성 차분이 수행됩니다.

In [ ]:
# 시나리오 A: 계절성 차분 적용
if SEASONAL and SEASONAL_D > 0:
    # 계절성 차분
    seasonal_diff_data = ts_data.diff(SEASONAL_PERIOD * SEASONAL_D).dropna()
    print(f"계절성 차분(주기={SEASONAL_PERIOD}, 차수={SEASONAL_D}) 데이터 생성 완료")
    
    # 계절성 + 일반 차분 조합 (필요시)
    if D > 0:
        combined_diff_data = seasonal_diff_data.diff(D).dropna()
        print(f"계절성 차분 + {D}차 차분 데이터 생성 완료")
    else:
        combined_diff_data = seasonal_diff_data
    
    # 계절성 차분 데이터 시각화
    plt.figure(figsize=(14, 10))
    
    plt.subplot(3, 1, 1)
    plt.plot(ts_data, label='원본 데이터')
    plt.title('원본 시계열 데이터')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 1, 2)
    plt.plot(seasonal_diff_data, label='계절성 차분')
    plt.title(f'계절성 차분 데이터 (주기={SEASONAL_PERIOD}, 차수={SEASONAL_D})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 1, 3)
    plt.plot(combined_diff_data, label='계절성 + 일반 차분')
    plt.title(f'계절성 + 일반 차분 데이터')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/arima_계절성차분.png')
    plt.show()
    
    # 계절성 차분 후 정상성 검정
    adf_seasonal_diff_result = adfuller(combined_diff_data.dropna())
    print(f'\n계절성 + 일반 차분 후 ADF 검정 결과:')
    print(f'ADF 통계량: {adf_seasonal_diff_result[0]:.4f}')
    print(f'p-value: {adf_seasonal_diff_result[1]:.4f}')
    
    if adf_seasonal_diff_result[1] < 0.05:
        print('계절성 + 일반 차분 후 시계열이 정상적일 가능성이 높습니다 (p < 0.05).')
        seasonal_diff_is_stationary = True
    else:
        print('계절성 + 일반 차분 후에도 시계열이 비정상적일 가능성이 높습니다 (p >= 0.05).')
        seasonal_diff_is_stationary = False

## 7. 데이터 분할
모델 학습과 성능 평가를 위해 데이터를 시간 순서에 따라 훈련(Train), 테스트(Test) 세트로 분할합니다.
- **훈련 세트**: 모델 학습에 사용됩니다.
- **테스트 세트**: 학습된 최종 모델의 일반화 성능을 평가하는 데 사용됩니다.
시계열 데이터는 과거 데이터로 미래를 예측하는 것이므로, 반드시 시간 순서를 유지하며 분할해야 합니다.

In [ ]:
# 훈련/테스트 분할
train_test_split_idx = int(len(ts_data) * TRAIN_SIZE)

train_data = ts_data[:train_test_split_idx]
test_data = ts_data[train_test_split_idx:]

print(f"훈련 데이터: {train_data.shape}, 기간: {train_data.index.min()} ~ {train_data.index.max()}")
print(f"테스트 데이터: {test_data.shape}, 기간: {test_data.index.min()} ~ {test_data.index.max()}")

# 훈련/테스트 데이터 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='테스트 데이터', color='red')
plt.axvline(x=train_data.index[-1], color='gray', linestyle='--')
plt.title('훈련/테스트 데이터 분할')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/arima_훈련테스트분할.png')
plt.show()

## 8. 모델 파라미터 선택 및 모델 구축
ARIMA/SARIMA 모델의 파라미터(p, d, q, P, D, Q, m)를 선택하고 모델을 구축합니다.
파라미터 선택은 ACF/PACF 분석이나 Auto ARIMA를 통해 수행할 수 있습니다.

### 8.1 모델 파라미터 선택 시나리오
ARIMA/SARIMA 모델의 파라미터(p, d, q, P, D, Q, m)를 선택하는 방법은 크게 두 가지 접근법이 있습니다:
1. **Auto ARIMA**: 정보 기준(AIC, BIC 등)을 최소화하여 최적의 파라미터를 자동으로 찾는 방법
2. **수동 파라미터 설정**: ACF/PACF 분석 및 도메인 지식을 바탕으로 사용자가 직접 파라미터를 지정하는 방법

아래에서는 두 가지 시나리오를 모두 제시합니다. 실제 분석 시에는 하나의 시나리오를 선택하여 활성화하세요.

#### 시나리오 A: Auto ARIMA를 이용한 최적 파라미터 탐색 (주석 처리됨)
`pmdarima` 라이브러리의 `auto_arima` 함수를 사용하면 AIC, BIC 등의 정보 기준을 최소화하는 최적의 (S)ARIMA 모델 파라미터를 자동으로 탐색할 수 있습니다.
현재 모델 파라미터 설정에서는 `USE_AUTO_ARIMA=False`이므로 이 시나리오는 실행되지 않습니다. 
필요시 1. 사용자 입력 파라미터 설정에서 `USE_AUTO_ARIMA=True`로 변경하거나, 아래 코드의 주석을 해제하여 사용하세요.

In [ ]:
# # 시나리오 A: Auto ARIMA (주석 처리됨)
# print("Auto ARIMA를 사용하여 최적 파라미터를 탐색합니다...")
# # auto_arima 모델 학습
# # seasonal=True로 설정하여 SARIMA 모델까지 고려, m은 계절성 주기
# # stepwise=True로 설정하여 효율적인 파라미터 탐색 수행
# auto_arima_model = pm.auto_arima(
#     train_data, 
#     start_p=1, start_q=1, 
#     max_p=3, max_q=3, # 탐색할 p, q 최대값
#     m=SEASONAL_PERIOD, # 계절성 주기
#     seasonal=SEASONAL, # 계절성 고려 여부
#     d=None, D=None, # 차분 차수는 자동 결정
#     trace=True, # 탐색 과정 출력
#     error_action='ignore',  # 오류 발생 시 무시
#     suppress_warnings=True, # 경고 메시지 숨김
#     stepwise=True # 단계적 탐색 수행
# )
# 
# print("\nAuto ARIMA 모델 요약:")
# print(auto_arima_model.summary())
# 
# # 최적 모델을 이후 단계에서 사용하기 위해 할당
# # auto_arima 결과는 SARIMAX 모델과 호환됩니다.
# model_fit = auto_arima_model
# best_order = auto_arima_model.order
# best_seasonal_order = auto_arima_model.seasonal_order
# print(f"\n최적 파라미터: order={best_order}, seasonal_order={best_seasonal_order}")

#### 시나리오 B: ARIMA 모델 수동 구축 및 학습 (활성화됨)
사용자가 직접 설정한 파라미터(p, d, q)를 사용하여 ARIMA 모델을 구축하고 학습합니다.
현재 모델 파라미터 설정에서는 `USE_AUTO_ARIMA=False`이므로 이 시나리오가 실행됩니다.

In [ ]:
# 시나리오 B: 수동 파라미터 ARIMA 모델
print(f"수동 파라미터 ARIMA({P},{D},{Q}) 모델을 구축합니다.")
model_arima = ARIMA(train_data, order=(P, D, Q))
model_arima_fit = model_arima.fit()
print("\nARIMA 모델 요약:")
print(model_arima_fit.summary())
model_fit = model_arima_fit # 이후 단계에서 사용할 모델

#### 시나리오 C: SARIMA 모델 수동 구축 및 학습 (활성화됨)
계절성이 있는 데이터의 경우, SARIMA 모델을 사용하여 계절 패턴을 더 잘 포착할 수 있습니다.
현재 모델 파라미터 설정에서는 `SEASONAL=True`이므로 이 시나리오가 실행됩니다.
계절성이 없는 데이터의 경우 `SEASONAL=False`로 설정하거나 아래 코드를 주석 처리하세요.

In [ ]:
# 시나리오 C: 계절성 고려 SARIMA 모델
print(f"\n수동 파라미터 SARIMA({P},{D},{Q})x({SEASONAL_P},{SEASONAL_D},{SEASONAL_Q},{SEASONAL_PERIOD}) 모델을 구축합니다.")
model_sarima = SARIMAX(
    train_data,
    order=(P, D, Q),
    seasonal_order=(SEASONAL_P, SEASONAL_D, SEASONAL_Q, SEASONAL_PERIOD)
)
model_sarima_fit = model_sarima.fit(disp=False)
print("\nSARIMA 모델 요약:")
print(model_sarima_fit.summary())

# 계절성 모델을 최종 모델로 설정 (ARIMA 대신 SARIMA 사용)
# 필요시 두 모델을 비교 후 최적 모델을 선택할 수 있습니다.
model_fit = model_sarima_fit
model_name = "SARIMA"  # 최종 선택된 모델 이름 (결과물 파일명 등에 사용)

## 9. 모델 예측 및 평가
구축한 모델을 사용하여 예측을 수행하고 성능을 평가합니다.

### 9.1 훈련 및 테스트 데이터 예측
학습된 모델을 사용하여 훈련 기간과 테스트 기간에 대한 예측값을 생성하고 실제값과 비교합니다.

In [ ]:
# 훈련 데이터 예측
pred_train = model_fit.predict(start=0, end=len(train_data)-1)

# 테스트 데이터 예측
pred_test = model_fit.predict(start=len(train_data), end=len(ts_data)-1) 
# 참고: forecast 메서드도 사용 가능 (다음 주석 해제)
# pred_test = model_fit.forecast(steps=len(test_data))
pred_test = pd.Series(pred_test, index=test_data.index)

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='테스트 데이터', color='red')
plt.plot(pred_train.index, pred_train.values, label='훈련 예측', color='green', linestyle='--')
plt.plot(pred_test.index, pred_test.values, label='테스트 예측', color='purple', linestyle='--')
plt.title(f'{model_name} 모델 예측')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/{model_name}_모델예측.png')
plt.show()

### 9.2 미래 예측 및 신뢰 구간
최종 모델을 사용하여 지정된 미래 기간(`FORECAST_HORIZON`)만큼 예측하고 신뢰 구간과 함께 시각화합니다.

In [ ]:
# 미래 예측
future_pred = model_fit.forecast(steps=FORECAST_HORIZON)
future_dates = pd.date_range(start=ts_data.index[-1], periods=FORECAST_HORIZON+1, freq=FREQUENCY)[1:]
future_pred = pd.Series(future_pred, index=future_dates)

# 신뢰 구간 계산
pred_ci = model_fit.get_forecast(steps=FORECAST_HORIZON).conf_int()
lower_ci = pd.Series(pred_ci.iloc[:, 0].values, index=future_dates)
upper_ci = pd.Series(pred_ci.iloc[:, 1].values, index=future_dates)

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(ts_data.index, ts_data.values, label='실제 데이터', color='blue')
plt.plot(future_dates, future_pred.values, label='미래 예측', color='red', linestyle='--')
plt.fill_between(future_dates, lower_ci.values, upper_ci.values, color='pink', alpha=0.3, label='95% 신뢰 구간')
plt.title(f'{model_name} 모델 미래 예측')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/{model_name}_미래예측.png')
plt.show()

### 9.3 모델 성능 평가
최종 선택된 모델의 테스트 세트에 대한 성능 지표를 계산하고 잔차 분석을 수행합니다.

In [ ]:
# 최종 모델 평가
final_mse = mean_squared_error(test_data, pred_test)
final_rmse = np.sqrt(final_mse)
final_mae = mean_absolute_error(test_data, pred_test)
final_r2 = r2_score(test_data, pred_test)

print(f"{model_name} 모델 평가:")
print(f"MSE: {final_mse:.4f}")
print(f"RMSE: {final_rmse:.4f}")
print(f"MAE: {final_mae:.4f}")
print(f"R²: {final_r2:.4f}")

# 잔차 분석
diag_fig = model_fit.plot_diagnostics(figsize=(14, 10))
diag_fig.suptitle(f'{model_name} 모델 잔차 분석', y=1.02)
diag_fig.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/{model_name}_잔차분석.png')
plt.show()

## 10. 결론 및 인사이트
이 노트북에서는 시계열 데이터에 ARIMA 및 SARIMA 모델을 적용하여 예측하는 과정을 살펴보았습니다.

In [ ]:
# 분석 요약 및 인사이트
print("\n" + "="*50)
print(f"{model_name} 모델 분석 요약")
print("="*50)

print(f"\n1. 데이터 정보:")
print(f"   - 데이터 기간: {ts_data.index.min()} ~ {ts_data.index.max()}")
print(f"   - 데이터 포인트 수: {len(ts_data)}")

print(f"\n2. 정상성 분석:")
stationarity = "정상적" if adf_result[1] < 0.05 else "비정상적"
print(f"   - 원본 데이터: {stationarity} (ADF p-value: {adf_result[1]:.4f})")

diff_stationarity = "정상적" if adf_diff_result[1] < 0.05 else "비정상적"
print(f"   - {D}차 차분 후: {diff_stationarity} (ADF p-value: {adf_diff_result[1]:.4f})")

print(f"\n3. 모델 파라미터:")
if model_name == "SARIMA":
    print(f"   - 비계절성 파라미터 (p,d,q): ({P},{D},{Q})")
    print(f"   - 계절성 파라미터 (P,D,Q,m): ({SEASONAL_P},{SEASONAL_D},{SEASONAL_Q},{SEASONAL_PERIOD})")
else:
    print(f"   - 파라미터 (p,d,q): ({P},{D},{Q})")

print(f"\n4. 모델 성능:")
print(f"   - RMSE: {final_rmse:.4f}")
print(f"   - MAE: {final_mae:.4f}")
print(f"   - R²: {final_r2:.4f}")

print("\n5. 주요 인사이트:")
print("   - ARIMA/SARIMA 모델은 시계열 데이터의 자기상관성과 이동평균 특성을 활용합니다.")
print("   - 계절성이 있는 데이터의 경우 SARIMA 모델이 더 적합할 수 있습니다.")
print("   - 정상성 확보를 위한 차분은 모델 성능에 중요한 영향을 미칩니다.")

print("\n6. 개선 방향:")
print("   - 외부 변수(날씨, 이벤트 등)를 추가하여 모델 성능 향상 가능")
print("   - 머신러닝 모델(Random Forest, XGBoost 등)과의 성능 비교")
print("   - 딥러닝 모델(LSTM 등)을 활용한 비선형 패턴 포착")

print("\nARIMA/SARIMA 모델 분석이 완료되었습니다!")